In [1]:
import os
import pandas as pd
import sys
sys.path.append("..")
from config import *

In [3]:
summary = []

for folder in os.listdir(RESISTANCE_RAW):
    folder_path = os.path.join(RESISTANCE_RAW, folder)
    if not os.path.isdir(folder_path):
        continue

    edx_file = None
    resistance_mean_file = None
    base_name = folder.replace("00", "", 1)

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            if "edx" in file.lower() and "similarity" not in file.lower():
                edx_file = os.path.join(folder_path, file)
            elif "resistance_mean" in file:
                resistance_mean_file = os.path.join(folder_path, file)

    if edx_file and resistance_mean_file:
        try:
            edx_df = pd.read_csv(edx_file)
            resistance_df = pd.read_csv(resistance_mean_file)

            # Drop 'Index' if exists
            if 'Index' in edx_df.columns:
                edx_df = edx_df.drop(columns=['Index'])

            # Keep only numeric columns
            edx_df = edx_df.select_dtypes(include='number')

            # Normalize EDX values row-wise
            edx_normalized = edx_df.div(edx_df.sum(axis=1), axis=0).reset_index(drop=True)

            # Extract x, y, R from resistance_mean file
            xy_r = resistance_df[['x', 'y', 'R']].copy()
            xy_r.rename(columns={'R': 'Resistance'}, inplace=True)
            xy_r.reset_index(drop=True, inplace=True)

            # Combine all
            merged = pd.concat([xy_r, edx_normalized], axis=1)
            merged.insert(0, 'ID', range(1, len(merged) + 1))

            # Reorder columns
            element_cols = edx_normalized.columns.tolist()
            final_cols = ['ID', 'x', 'y'] + element_cols + ['Resistance']
            merged = merged[final_cols]

            # Save file with new name
            elements_str = "-".join(element_cols)
            output_name = f"{base_name}_{elements_str}_RT.csv"
            output_path = os.path.join(RESISTANCE_CLEANED_FILES, output_name)
            merged.to_csv(output_path, index=False)
            

            # Save min/max element indices
            for col in edx_normalized.columns:
                summary.append({
                    "Folder": folder,
                    "Element": col,
                    "MinIndex": edx_normalized[col].idxmin(),
                    "MaxIndex": edx_normalized[col].idxmax()
                })

        except Exception as e:
            print(f" Error processing {folder}: {e}")

# Save min/max index summary
summary_df = pd.DataFrame(summary)
summary_df.to_csv(os.path.join(RESISTANCE_RAW, "EDX_min_max_summary.csv"), index=False)
print("\n Min/Max index summary saved as: EDX_min_max_summary.csv")


 Min/Max index summary saved as: EDX_min_max_summary.csv
